In [1]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
from crow.cluster_modules.shear_profile import *
from crow.recipes.murata_binned_spec_z_grid_real import MurataBinnedSpecZRecipeGrid
from crow.recipes.murata_binned_spec_z import MurataBinnedSpecZRecipe
from crow.cluster_modules.mass_proxy import MurataBinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
from crow.cluster_modules.purity import PurityAguena16
from crow.cluster_modules.completeness import CompletenessAguena16
#from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time
import numpy as np

from scipy.integrate import dblquad, tplquad, simpson

In [2]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma = ClusterShearProfile(cosmo, hmf, 4.0, True)
pivot_mass, pivot_redshift = 14.625862906, 0.6
comp_dist = CompletenessAguena16()
pur_dist = PurityAguena16()
mass_distribution = MurataBinned(pivot_mass, pivot_redshift)#, pur_dist)
redshift_distribution = SpectroscopicRedshift()



In [3]:
##### Parameters to be used in both recipes #####
mass_points = 100
redshift_points = 30
log_proxy_points = 20
sky_area = 440
mass_interval = (12.0, 17.0)
cluster_theory = cl_delta_sigma
z_bin = (0.2, 0.4)
z_points = np.linspace(z_bin[0], z_bin[1], redshift_points) 
proxy_bin = (1.0, 1.3)
proxy_points = np.linspace(proxy_bin[0], proxy_bin[1], log_proxy_points)
radius_center = 4.0
#################################################

recipe_integral = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
    )

recipe_grid = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
    )

## Testing mass function

In [4]:
hmf_grid = recipe_grid.compute_hmf_grid(z_points, sky_area)
log_mass_grid = recipe_grid.log_mass_grid
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    # 1. Convert scalars to single-element arrays to satisfy the methods
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])

    # 2. Call the methods with the correct, named arrays
    # comoving_volume(z, sky_area)
    vol_array = recipe_integral.cluster_theory.comoving_volume(z_array, sky_area)
    
    # mass_function(log_mass, z)
    hmf_array = recipe_integral.cluster_theory.mass_function(log_mass_array, z_array)
    
    # 3. Return the result as a scalar float
    return (vol_array * hmf_array)[0]



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=hmf_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 268878.88767182204, dblquad integral 268878.7658501723
Abs error 0.12182164972182363, rel error 4.530727792939615e-07


## Testing mass-richness

In [5]:
mass_richness_grid = recipe_grid.compute_mass_richness_grid(z_points, proxy_points)
###################3
print(f"BECAUSE MASS RICHNESS INTEGRAL HAS PURITY INSIDE, INSTANTIATE A NEW ONE FOR THIS TEST")
mass_distribution_pure = MurataBinned(pivot_mass, pivot_redshift)
recipe_integral_pure = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
    )
################################
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    return recipe_integral.mass_distribution.distribution(log_mass_array, z_array, proxy_bin)    

z_grid_mesh, log_mass_grid_mesh = np.meshgrid(
            z_points, log_mass_grid, indexing='ij'
        )




integral_mass_richness, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)

integral_over_mass = simpson(
    y=mass_richness_grid, 
    x=log_mass_grid, 
    axis=2
)

integral_over_proxy = simpson(
    y=integral_over_mass, 
    x=proxy_points * np.log(10.0), 
    axis=0
)

simpson_mass_richness = simpson(
    y=integral_over_proxy, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_mass_richness}, dblquad integral {integral_mass_richness}")
print(f"Abs error {abs(integral_mass_richness - simpson_mass_richness)}, rel error {abs(1.0 - simpson_mass_richness/integral_mass_richness)}")



BECAUSE MASS RICHNESS INTEGRAL HAS PURITY INSIDE, INSTANTIATE A NEW ONE FOR THIS TEST
Simpson Integral 0.07500000000000002, dblquad integral 0.07500000000000012
Abs error 9.71445146547012e-17, rel error 1.3322676295501878e-15


## Testing completeness

In [6]:
comp_grid = recipe_grid.compute_completeness_grid(z_points)
def integrand(log_mass_scalar, z_scalar):
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    return recipe_integral.completeness_distribution(log_mass_array, z_array)



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=comp_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 0.6853199859674308, dblquad integral 0.6853199864484147
Abs error 4.809839193598009e-10, rel error 7.018384762247365e-10


## Testing Purity

In [7]:
# pur_grid = recipe_grid.compute_purity_grid(z_points, proxy_points)
# def integrand(ln_proxy_scalar, z_scalar):
#     log_proxy_scalar = ln_proxy_scalar / np.log(10.0)
#     z_array = np.array([z_scalar])
#     log_proxy_array = np.array([log_proxy_scalar])
#     return recipe_integral.mass_distribution.purity.distribution(z_array, log_proxy_array)



# integral_hmf, error_estimate = dblquad(
#     func=integrand, 
#     a=z_bin[0],       # lower limit for z (x-axis)
#     b=z_bin[1],       # upper limit for z (x-axis)
#     gfun=lambda x: proxy_bin[0]  * np.log(10.0), # lower limit for log_mass (y-axis)
#     hfun=lambda x: proxy_bin[1] * np.log(10.0), # upper limit for log_mass (y-axis)
# )
# integral_over_proxy = simpson(
#     y=pur_grid, 
#     x=proxy_points, 
#     axis=1
# )

# # Step B: Integrate over the redshift dimension (axis=0 of the new array)
# # The result is the final scalar integrated number count
# simpson_hmf = simpson(
#     y=integral_over_proxy * np.log(10.0), 
#     x=z_points, 
#     axis=0
# )

# print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
# print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



## Testing Shear

In [8]:
shear_grid = recipe_grid.compute_shear_grid(z_points, radius_center)
log_mass_grid = recipe_grid.log_mass_grid
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    # 1. Convert scalars to single-element arrays to satisfy the methods
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    shear = recipe_integral.cluster_theory.compute_shear_profile(log_mass_array, z_array, radius_center = radius_center)
    
    # 3. Return the result as a scalar float
    return shear[0]



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=shear_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 97566729539515.73, dblquad integral 97566654129403.34
Abs error 75410112.390625, rel error 7.729086650698491e-07


## Testing Counts

In [9]:
recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral_pure.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 465.89130908891366, dblquad integral 465.89348909430385
Abs error 0.002180005390187034, rel error 4.679192650680619e-06
First eval took: 0.007207155227661133, second eval took: 0.0014400482177734375, integral took: 0.07780647277832031
After reset: 0.0068874359130859375
 Counts 1, 2 ,3: (np.float64(465.89130908891366), np.float64(465.89130908891366), np.float64(465.89130908891366))


## Testing Shear Profile

In [10]:
average_on = ClusterProperty.NONE
average_on |= ClusterProperty.DELTASIGMA
recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center,sky_area, average_on)
t2 = time.time()
counts_integral = recipe_integral_pure.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center, sky_area,average_on)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center, sky_area,average_on)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center, sky_area,average_on)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


Simpson Integral 3674727743264800.0, dblquad integral 3832214530946015.0
Abs error 157486787681215.0, rel error 0.041095504025015495
First eval took: 1.305562973022461, second eval took: 0.0015537738800048828, integral took: 0.5741713047027588
After reset: 1.3940379619598389
 Counts 1, 2 ,3: (np.float64(3674727743264800.0), np.float64(3674727743264800.0), np.float64(3674727743264800.0))


## Test speed


In [11]:
mass_distribution_pure = MurataBinned(pivot_mass, pivot_redshift)


##### Parameters to be used in both recipes #####
mass_points = 101
redshift_points = 40
log_proxy_points = 40
sky_area = 440
mass_interval = (12.0, 17.0)
cluster_theory = cl_delta_sigma
z_bin = (0.2, 0.4)
z_points = np.linspace(z_bin[0], z_bin[1], redshift_points) 
proxy_bin = (1.0, 1.3)
proxy_points = np.linspace(proxy_bin[0], proxy_bin[1], log_proxy_points)
radius_center = 4.0
average_on_counts = ClusterProperty.NONE
average_on_shear = ClusterProperty.NONE
average_on_shear |= ClusterProperty.DELTASIGMA
#################################################

recipe_integral_speed = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
    )

recipe_grid_speed = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
    )
recipe_grid_speed.reset_grids_cache()
counts_grid = recipe_grid_speed.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area,average_on_counts)
counts_integral = recipe_integral_speed.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_counts)

counts_grid = recipe_grid_speed.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center,sky_area, average_on_shear)
t2 = time.time()
counts_integral = recipe_integral_speed.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center, sky_area,average_on_shear)
t21 = time.time()


In [12]:
import time
import numpy as np

# Your parameters
mass_distribution_pure = MurataBinned(pivot_mass, pivot_redshift)

mass_points = 101
redshift_points = 20
log_proxy_points = 20
sky_area = 440
mass_interval = (12.0, 17.0)
z_bin = (0.2, 0.4)
proxy_bin = (1.0, 1.3)
radius_center = 4.0

average_on_counts = ClusterProperty.NONE
average_on_shear  = ClusterProperty.DELTASIGMA

# Recipes
recipe_integral_speed = MurataBinnedSpecZRecipe(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution_pure,
    completeness=comp_dist,
)

recipe_grid_speed = MurataBinnedSpecZRecipeGrid(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution_pure,
    completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
)

# ---- One-shot timing: counts ----
recipe_grid_speed.reset_grids_cache()

t0 = time.time()
counts_grid = recipe_grid_speed.evaluate_theory_prediction_counts(
    z_bin, proxy_bin, sky_area, average_on_counts
)
t1 = time.time()

counts_integral = recipe_integral_speed.evaluate_theory_prediction_counts(
    z_bin, proxy_bin, sky_area, average_on_counts
)
t2 = time.time()

print("\n--- One-shot counts ---")
print("Grid time:    %.4f s" % (t1 - t0))
print("Integral time: %.4f s" % (t2 - t1))


# ---- One-shot timing: shear ----
t0 = time.time()
shear_grid = recipe_grid_speed.evaluate_theory_prediction_shear_profile(
    z_bin, proxy_bin, radius_center, sky_area, average_on_shear
)
t1 = time.time()

shear_integral = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
    z_bin, proxy_bin, radius_center, sky_area, average_on_shear
)
t2 = time.time()

print("\n--- One-shot shear ---")
print("Grid time:    %.4f s" % (t1 - t0))
print("Integral time: %.4f s" % (t2 - t1))



--- One-shot counts ---
Grid time:    0.2770 s
Integral time: 0.0027 s

--- One-shot shear ---
Grid time:    0.9574 s
Integral time: 0.5117 s


In [13]:
richness_bins = [(0.3,0.4), (0.4,0.6), (0.6,0.8), (0.8,1.0)]   # example
proxy_bins    = [(1.0,1.1), (1.1,1.2), (1.2,1.3), (1.3,1.4), (1.4,1.5)]
radii         = np.linspace(1.0, 10.0, 4)

# ----------------------------------------------
# Total GRID computation time
# ----------------------------------------------
recipe_grid_speed.reset_grids_cache()
counts_grid_arr = []
shear_grid_arr = []
tg0 = time.time()
for rbin in richness_bins:
    for pbin in proxy_bins:
        for R in radii:
            counts_grid_arr.append(recipe_grid_speed.evaluate_theory_prediction_counts(rbin, pbin, sky_area, average_on_counts))
            shear_grid_arr.append(recipe_grid_speed.evaluate_theory_prediction_shear_profile(rbin, pbin, R, sky_area, average_on_shear))
tg1 = time.time()
print("\nTOTAL GRID TIME = %.2f s" % (tg1 - tg0))


# ----------------------------------------------
# Total INTEGRAL computation time
# ----------------------------------------------
counts_int_arr = []
shear_int_arr = []
ti0 = time.time()
for rbin in richness_bins:
    for pbin in proxy_bins:
        for R in radii:
            counts_int_arr.append(recipe_integral_speed.evaluate_theory_prediction_counts(rbin, pbin, sky_area, average_on_counts))
            shear_int_arr.append(recipe_integral_speed.evaluate_theory_prediction_shear_profile(rbin, pbin, R, sky_area, average_on_shear))
ti1 = time.time()
print("TOTAL INTEGRAL TIME = %.2f s" % (ti1 - ti0))

# ----------------------------------------------
# Comparison of Results
# ----------------------------------------------

# Convert lists to NumPy arrays for easy calculation
counts_grid = np.array(counts_grid_arr)
counts_int = np.array(counts_int_arr)
shear_grid = np.array(shear_grid_arr)
shear_int = np.array(shear_int_arr)

counts_diff = np.where(
    counts_int != 0.0, 
    np.abs(counts_grid - counts_int) / np.abs(counts_int), 
    0.0
)
rms_counts_diff = np.sqrt(np.mean(counts_diff**2))

shear_diff = np.where(
    shear_int != 0.0, 
    np.abs(shear_grid - shear_int) / np.abs(shear_int), 
    0.0
)
rms_shear_diff = np.sqrt(np.mean(shear_diff**2))

# Final Comparison Printout
print("\n--- RESULTS COMPARISON ---")
print("Total number of bins computed: %d" % len(counts_grid))
print("RMS Relative Diff (Counts): %.4e" % rms_counts_diff)
print("RMS Relative Diff (Shear):  %.4e" % rms_shear_diff)
print("--------------------------")



TOTAL GRID TIME = 14.76 s
TOTAL INTEGRAL TIME = 29.54 s

--- RESULTS COMPARISON ---
Total number of bins computed: 80
RMS Relative Diff (Counts): 4.3241e-06
RMS Relative Diff (Shear):  2.6710e-02
--------------------------


In [14]:
print(average_on_shear)

ClusterProperty.DELTASIGMA


In [15]:
def frac_diff(a, b):
    return np.abs(a - b) / (np.abs(b) + 1e-12)


print("\n===== BEGIN ACCURACY SWEEP WITH COUNTS =====\n")

# --- ranges to test ---
proxy_sweep    = [2, 5, 10, 20, 30, 40, 60, 80, 120, 160]
z_sweep        = [2, 5, 10, 20, 30, 40]
mass_sweep     = [5, 10, 20, 40, 60, 80, 120]

# --- fixed test bin ---
z_lo, z_hi = z_bin
proxy_lo, proxy_hi = proxy_bin
R = radius_center

print("Test bin:")
print("  z = (%.3f, %.3f)" % z_bin)
print("  proxy = (%.3f, %.3f)" % proxy_bin)
print("  R = %.3f" % R)


# --------------------------------------------------------------
# 1) PROXY SWEEP
# --------------------------------------------------------------
print("\n---- Sweep log_proxy_points ----")

for pts in proxy_sweep:
    print(f"\nTrying log_proxy_points = {pts}")

    recipe_grid_speed_pts = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
        log_proxy_points=pts,
        redshift_points=redshift_points,
        log_mass_points=mass_points,
    )
    recipe_grid_speed_pts.reset_grids_cache()
    
    # Shear evaluation
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    
    # Counts evaluation
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    recipe_grid_speed_pts.reset_grids_cache()
    
    # Reference shear
    ref_shear = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    
    # Reference counts
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = float(frac_diff(val_shear, ref_shear))
    diff_counts = float(frac_diff(counts_val, ref_counts))

    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    if diff_shear < 0.01 and diff_counts < 0.01:
        print("  → Achieved ≤1% accuracy at log_proxy_points =", pts)
        break


# --------------------------------------------------------------
# 2) REDSHIFT SWEEP
# --------------------------------------------------------------
print("\n---- Sweep redshift_points ----")

for pts in z_sweep:
    print(f"\nTrying redshift_points = {pts}")

    recipe_grid_speed_pts = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
        log_proxy_points=log_proxy_points,
        redshift_points=pts,
        log_mass_points=mass_points,
    )
    recipe_grid_speed_pts.reset_grids_cache()
    
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )

    ref_shear = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = float(frac_diff(val_shear, ref_shear))
    diff_counts = float(frac_diff(counts_val, ref_counts))
    
    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    if diff_shear < 0.01 and diff_counts < 0.01:
        print("  → Achieved ≤1% accuracy at redshift_points =", pts)
        break


# --------------------------------------------------------------
# 3) MASS SWEEP
# --------------------------------------------------------------
print("\n---- Sweep log_mass_points ----")

for pts in mass_sweep:
    print(f"\nTrying log_mass_points = {pts}")

    recipe_grid_speed_pts = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
        log_proxy_points=log_proxy_points,
        redshift_points=redshift_points,
        log_mass_points=pts,
    )
    recipe_grid_speed_pts.reset_grids_cache()
    
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_counts(
        z_bin, proxy_bin,  sky_area
    )

    ref_shear = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = float(frac_diff(val_shear, ref_shear))
    diff_counts = float(frac_diff(counts_val, ref_counts))
    
    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    # Optional early break if both ≤1%
    # if diff_shear < 0.01 and diff_counts < 0.01:
    #     print("  → Achieved ≤1% accuracy at log_mass_points =", pts)
    #     break


print("\n===== END ACCURACY SWEEP WITH COUNTS =====\n")



===== BEGIN ACCURACY SWEEP WITH COUNTS =====

Test bin:
  z = (0.200, 0.400)
  proxy = (1.000, 1.300)
  R = 4.000

---- Sweep log_proxy_points ----

Trying log_proxy_points = 2
  shear diff  = 0.032592
  counts diff = 0.121889

Trying log_proxy_points = 5
  shear diff  = 0.041111
  counts diff = 0.000032

Trying log_proxy_points = 10
  shear diff  = 0.041094
  counts diff = 0.000012

Trying log_proxy_points = 20
  shear diff  = 0.041096
  counts diff = 0.000005

Trying log_proxy_points = 30
  shear diff  = 0.041096
  counts diff = 0.000005

Trying log_proxy_points = 40
  shear diff  = 0.041096
  counts diff = 0.000005

Trying log_proxy_points = 60
  shear diff  = 0.041096
  counts diff = 0.000005

Trying log_proxy_points = 80
  shear diff  = 0.041096
  counts diff = 0.000005

Trying log_proxy_points = 120
  shear diff  = 0.041096
  counts diff = 0.000005

Trying log_proxy_points = 160
  shear diff  = 0.041096
  counts diff = 0.000005

---- Sweep redshift_points ----

Trying redshift_p

In [16]:
print(np.linspace(1,10, 5))

[ 1.    3.25  5.5   7.75 10.  ]
